In [1]:
# 좌우 / 시선(face_mesh) 측정

import cv2
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Webcam not opened.")
    exit()

print("🎥 FaceMesh started - ESC to exit")

# 랜드마크 인덱스
LEFT_EYE_OUTER = 33
RIGHT_EYE_OUTER = 263
LEFT_IRIS_LEFT = 471
RIGHT_IRIS_RIGHT = 474

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    gaze_text = ""

    if result.multi_face_landmarks:
        face = result.multi_face_landmarks[0]
        lm = face.landmark

        # 좌표 추출 (픽셀 단위)
        left_eye_outer = np.array([lm[LEFT_EYE_OUTER].x * w, lm[LEFT_EYE_OUTER].y * h])
        left_iris_left = np.array([lm[LEFT_IRIS_LEFT].x * w, lm[LEFT_IRIS_LEFT].y * h])

        right_eye_outer = np.array([lm[RIGHT_EYE_OUTER].x * w, lm[RIGHT_EYE_OUTER].y * h])
        right_iris_right = np.array([lm[RIGHT_IRIS_RIGHT].x * w, lm[RIGHT_IRIS_RIGHT].y * h])

        # 거리 계산
        left_distance = np.linalg.norm(left_eye_outer - left_iris_left)
        right_distance = np.linalg.norm(right_eye_outer - right_iris_right)

        # 시선 판별 기준
        if left_distance < 10:
            gaze_text = "👁️ Looking LEFT"
        elif right_distance < 10:
            gaze_text = "👁️ Looking RIGHT"
        else:
            gaze_text = "👁️ Looking CENTER"

        # 디버깅용 점 시각화
        for idx in [LEFT_EYE_OUTER, LEFT_IRIS_LEFT, RIGHT_EYE_OUTER, RIGHT_IRIS_RIGHT]:
            cx = int(lm[idx].x * w)
            cy = int(lm[idx].y * h)
            cv2.circle(frame, (cx, cy), 3, (0, 255, 255), -1)

    # 출력 텍스트
    cv2.putText(frame, gaze_text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    cv2.imshow("Iris Gaze Tracking (Outer Iris Landmarks)", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

🎥 FaceMesh started - ESC to exit


In [1]:
# 좌우 / 시선(face_mesh), 고개(face_mesh), 어깨(Pose) 측정 

import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
pose = mp_pose.Pose(static_image_mode=False)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Webcam not opened.")
    exit()

print("🎥 Tracking started - ESC to exit")

# 랜드마크 인덱스
LEFT_EYE_OUTER = 33
RIGHT_EYE_OUTER = 263
LEFT_IRIS_LEFT = 471
RIGHT_IRIS_RIGHT = 474
LEFT_EAR = 234
RIGHT_EAR = 454
LEFT_EYE_CENTER = 468
RIGHT_EYE_CENTER = 473

LEFT_SHOULDER = mp_pose.PoseLandmark.LEFT_SHOULDER
RIGHT_SHOULDER = mp_pose.PoseLandmark.RIGHT_SHOULDER

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    face_result = face_mesh.process(rgb)
    pose_result = pose.process(rgb)

    gaze_text, head_text, shoulder_text = "", "", ""

    # ---------- 1. 얼굴 방향, 시선 ----------
    if face_result.multi_face_landmarks:
        face = face_result.multi_face_landmarks[0]
        lm = face.landmark

        # 시선 방향
        left_eye_outer = np.array([lm[LEFT_EYE_OUTER].x * w, lm[LEFT_EYE_OUTER].y * h])
        left_iris = np.array([lm[LEFT_IRIS_LEFT].x * w, lm[LEFT_IRIS_LEFT].y * h])
        right_eye_outer = np.array([lm[RIGHT_EYE_OUTER].x * w, lm[RIGHT_EYE_OUTER].y * h])
        right_iris = np.array([lm[RIGHT_IRIS_RIGHT].x * w, lm[RIGHT_IRIS_RIGHT].y * h])

        left_dist = np.linalg.norm(left_eye_outer - left_iris)
        right_dist = np.linalg.norm(right_eye_outer - right_iris)

        if left_dist < 10:
            gaze_text = "👁️ Gaze: LEFT"
        elif right_dist < 10:
            gaze_text = "👁️ Gaze: RIGHT"
        else:
            gaze_text = "👁️ Gaze: CENTER"

        # 고개 방향 (양 눈 ↔ 귀 거리 비교)
        left_eye_center = lm[LEFT_EYE_CENTER]
        right_eye_center = lm[RIGHT_EYE_CENTER]
        left_ear = lm[LEFT_EAR]
        right_ear = lm[RIGHT_EAR]

        left_eye_to_ear = abs(left_eye_center.x - left_ear.x)
        right_eye_to_ear = abs(right_eye_center.x - right_ear.x)

        if left_eye_to_ear > right_eye_to_ear + 0.02:
            head_text = "🧠 Head: RIGHT TURN"
        elif right_eye_to_ear > left_eye_to_ear + 0.02:
            head_text = "🧠 Head: LEFT TURN"
        else:
            head_text = "🧠 Head: CENTER"

        # 눈/홍채 시각화
        for idx in [LEFT_EYE_OUTER, LEFT_IRIS_LEFT, RIGHT_EYE_OUTER, RIGHT_IRIS_RIGHT]:
            cx, cy = int(lm[idx].x * w), int(lm[idx].y * h)
            cv2.circle(frame, (cx, cy), 3, (0, 255, 255), -1)

    # ---------- 2. 어깨 자세 ----------
    if pose_result.pose_landmarks:
        landmarks = pose_result.pose_landmarks.landmark
        left_y = landmarks[LEFT_SHOULDER].y
        right_y = landmarks[RIGHT_SHOULDER].y
        diff = left_y - right_y

        if diff > 0.018:
            shoulder_text = "🧍 Shoulder: LEFT UP"
        elif diff < -0.018:
            shoulder_text = "🧍 Shoulder: RIGHT UP"
        else:
            shoulder_text = "🧍 Shoulder: STRAIGHT"

        mp_drawing.draw_landmarks(
            frame,
            pose_result.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3)
        )

    # ---------- 전체 텍스트 출력 ----------
    y0 = 30
    for line in [gaze_text, head_text, shoulder_text]:
        if line:
            cv2.putText(frame, line, (10, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            y0 += 30

    cv2.imshow("Full Tracker (Gaze + Head + Shoulder)", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

🎥 Tracking started - ESC to exit


In [9]:
# 시선 측정할때, 뒤로가면 수치가 작아지니까 처음 if인 왼쪽이 계속 떠서, 눈동자 길이 생각해서 눈 왼쪽과 얼마나 가까운지 옆에 따는 방식으로 해봄.
import cv2
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 랜드마크 인덱스
LEFT_EYE_OUTER = 33
LEFT_IRIS_LEFT = 471
LEFT_IRIS_RIGHT = 469

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Webcam not opened.")
    exit()

print("🎥 Gaze ratio tracking started - ESC to exit")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    gaze_text = ""

    if result.multi_face_landmarks:
        face = result.multi_face_landmarks[0]
        lm = face.landmark

        # 픽셀 좌표 추출
        eye_outer = np.array([lm[LEFT_EYE_OUTER].x * w, lm[LEFT_EYE_OUTER].y * h])
        iris_left = np.array([lm[LEFT_IRIS_LEFT].x * w, lm[LEFT_IRIS_LEFT].y * h])
        iris_right = np.array([lm[LEFT_IRIS_RIGHT].x * w, lm[LEFT_IRIS_RIGHT].y * h])

        iris_width = np.linalg.norm(iris_right - iris_left) + 1e-6  # 나눗셈 오류 방지
        to_outer = np.linalg.norm(iris_left - eye_outer)

        ratio = to_outer / iris_width

        # 임계값 기준 (0.48 이하면 왼쪽 응시로 판단)
        if ratio < 0.48:
            gaze_text = f"👁️ LEFT EYE → Looking LEFT ({ratio:.2f})"
        else:
            gaze_text = f"👁️ LEFT EYE → CENTER/RIGHT ({ratio:.2f})"

        # 디버깅용 점 출력
        for idx in [LEFT_EYE_OUTER, LEFT_IRIS_LEFT, LEFT_IRIS_RIGHT]:
            cx, cy = int(lm[idx].x * w), int(lm[idx].y * h)
            cv2.circle(frame, (cx, cy), 3, (0, 255, 255), -1)

    # 텍스트 출력
    cv2.putText(frame, gaze_text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    cv2.imshow("Gaze Ratio (Left Eye)", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

🎥 Gaze ratio tracking started - ESC to exit


In [ ]:
# 시선, 고개, 어깨 분석 (시선 비율로 해서)합쳤는데, 고개가 잘 안됌 -> 확인해야함. 
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

pose = mp_pose.Pose(static_image_mode=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# 랜드마크 인덱스
LEFT_EYE_OUTER = 33
LEFT_IRIS_LEFT = 471
LEFT_IRIS_RIGHT = 469

RIGHT_EYE_OUTER = 263
RIGHT_IRIS_LEFT = 476
RIGHT_IRIS_RIGHT = 474

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Webcam not opened.")
    exit()

print("🎥 Tracking started - ESC to exit")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 시선 분석
    gaze_text = ""
    face_result = face_mesh.process(rgb)
    if face_result.multi_face_landmarks:
        face = face_result.multi_face_landmarks[0]
        lm = face.landmark

        # 왼쪽 눈
        left_eye_outer = np.array([lm[LEFT_EYE_OUTER].x * w, lm[LEFT_EYE_OUTER].y * h])
        left_iris_left = np.array([lm[LEFT_IRIS_LEFT].x * w, lm[LEFT_IRIS_LEFT].y * h])
        left_iris_right = np.array([lm[LEFT_IRIS_RIGHT].x * w, lm[LEFT_IRIS_RIGHT].y * h])
        left_iris_width = np.linalg.norm(left_iris_right - left_iris_left) + 1e-6
        left_ratio = np.linalg.norm(left_iris_left - left_eye_outer) / left_iris_width

        # 오른쪽 눈
        right_eye_outer = np.array([lm[RIGHT_EYE_OUTER].x * w, lm[RIGHT_EYE_OUTER].y * h])
        right_iris_left = np.array([lm[RIGHT_IRIS_LEFT].x * w, lm[RIGHT_IRIS_LEFT].y * h])
        right_iris_right = np.array([lm[RIGHT_IRIS_RIGHT].x * w, lm[RIGHT_IRIS_RIGHT].y * h])
        right_iris_width = np.linalg.norm(right_iris_right - right_iris_left) + 1e-6
        right_ratio = np.linalg.norm(right_eye_outer - right_iris_right) / right_iris_width

        # 시선 방향 결정
        left_gaze = "LEFT" if left_ratio < 0.48 else "CENTER"
        right_gaze = "RIGHT" if right_ratio < 0.48 else "CENTER"
        result_gaze = "LEFT" if left_gaze == "LEFT" else "RIGHT" if right_gaze == "RIGHT" else "CENTER"

        gaze_text = f"L-Gaze:{left_gaze}({left_ratio:.2f})|R-Gaze:{right_gaze}({right_ratio:.2f})→{result_gaze}"

        # 디버깅용 눈 주변 점
        for idx in [LEFT_EYE_OUTER, LEFT_IRIS_LEFT, LEFT_IRIS_RIGHT,
                    RIGHT_EYE_OUTER, RIGHT_IRIS_LEFT, RIGHT_IRIS_RIGHT]:
            cx, cy = int(lm[idx].x * w), int(lm[idx].y * h)
            cv2.circle(frame, (cx, cy), 3, (0, 255, 255), -1)

    # 포즈 분석
    pose_result = pose.process(rgb)
    pose_text = ""
    if pose_result.pose_landmarks:
        landmarks = pose_result.pose_landmarks.landmark

        # 고개 방향 (좌우 귀 기준)
        left_ear_x = landmarks[mp_pose.PoseLandmark.LEFT_EAR].x
        right_ear_x = landmarks[mp_pose.PoseLandmark.RIGHT_EAR].x
        head_text = ""
        if left_ear_x - right_ear_x > 0.08:
            head_text = "HEAD → LEFT"
        elif right_ear_x - left_ear_x > 0.08:
            head_text = "HEAD → RIGHT"
        else:
            head_text = "HEAD → CENTER"

        # 어깨 높이 비교
        left_shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y
        right_shoulder_y = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y
        diff = left_shoulder_y - right_shoulder_y
        if diff > 0.012:
            shoulder_text = "LEFT SHOULDER UP"
        elif diff < -0.012:
            shoulder_text = "RIGHT SHOULDER UP"
        else:
            shoulder_text = "SHOULDERS STRAIGHT"

        pose_text = f"{head_text} | 🧍 {shoulder_text}"

        # 포즈 랜드마크 그리기
        mp_drawing.draw_landmarks(
            frame,
            pose_result.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

    # 결과 텍스트 출력
    y = 30
    for line in [gaze_text, pose_text]:
        if line:
            cv2.putText(frame, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            y += 30

    cv2.imshow("Full Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

🎥 Tracking started - ESC to exit


In [3]:
# 좌우 시선, 고개, 어깨 완성

import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

pose = mp_pose.Pose(static_image_mode=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# FaceMesh & Pose 랜드마크 인덱스
LEFT_EYE_OUTER = 33
LEFT_IRIS_LEFT = 471
LEFT_IRIS_RIGHT = 469
LEFT_EYE_CENTER = 468

RIGHT_EYE_OUTER = 263
RIGHT_IRIS_LEFT = 476
RIGHT_IRIS_RIGHT = 474
RIGHT_EYE_CENTER = 473

LEFT_EAR = mp_pose.PoseLandmark.LEFT_EAR
RIGHT_EAR = mp_pose.PoseLandmark.RIGHT_EAR
LEFT_SHOULDER = mp_pose.PoseLandmark.LEFT_SHOULDER
RIGHT_SHOULDER = mp_pose.PoseLandmark.RIGHT_SHOULDER

# 웹캠 시작
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Webcam not opened.")
    exit()

print("🎥 Gaze + Head + Shoulder Tracking Started - ESC to exit")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # FaceMesh 시선 분석
    gaze_text = ""
    head_text = ""
    face_result = face_mesh.process(rgb)

    # Pose 분석
    pose_result = pose.process(rgb)

    if face_result.multi_face_landmarks:
        face = face_result.multi_face_landmarks[0]
        lm = face.landmark

        # === 왼쪽 눈 시선 분석 ===
        left_eye_outer = np.array([lm[LEFT_EYE_OUTER].x * w, lm[LEFT_EYE_OUTER].y * h])
        left_iris_left = np.array([lm[LEFT_IRIS_LEFT].x * w, lm[LEFT_IRIS_LEFT].y * h])
        left_iris_right = np.array([lm[LEFT_IRIS_RIGHT].x * w, lm[LEFT_IRIS_RIGHT].y * h])
        left_iris_width = np.linalg.norm(left_iris_right - left_iris_left) + 1e-6
        left_ratio = np.linalg.norm(left_iris_left - left_eye_outer) / left_iris_width

        # === 오른쪽 눈 시선 분석 ===
        right_eye_outer = np.array([lm[RIGHT_EYE_OUTER].x * w, lm[RIGHT_EYE_OUTER].y * h])
        right_iris_left = np.array([lm[RIGHT_IRIS_LEFT].x * w, lm[RIGHT_IRIS_LEFT].y * h])
        right_iris_right = np.array([lm[RIGHT_IRIS_RIGHT].x * w, lm[RIGHT_IRIS_RIGHT].y * h])
        right_iris_width = np.linalg.norm(right_iris_right - right_iris_left) + 1e-6
        right_ratio = np.linalg.norm(right_eye_outer - right_iris_right) / right_iris_width

        # 시선 방향 결정
        left_gaze = "LEFT" if left_ratio < 0.48 else "CENTER"
        right_gaze = "RIGHT" if right_ratio < 0.48 else "CENTER"
        result_gaze = "LEFT" if left_gaze == "LEFT" else "RIGHT" if right_gaze == "RIGHT" else "CENTER"

        gaze_text = f"L-Gaze:{left_gaze}({left_ratio:.2f})|R-Gaze:{right_gaze}({right_ratio:.2f})→{result_gaze}"

        # === 고개 방향: 양 눈 ↔ 귀 거리 비교 ===
        if pose_result.pose_landmarks:
            pose_lm = pose_result.pose_landmarks.landmark

            left_eye_center = lm[LEFT_EYE_CENTER]
            right_eye_center = lm[RIGHT_EYE_CENTER]
            left_ear = pose_lm[LEFT_EAR]
            right_ear = pose_lm[RIGHT_EAR]

            left_eye_to_ear = abs(left_eye_center.x - left_ear.x)
            right_eye_to_ear = abs(right_eye_center.x - right_ear.x)

            if left_eye_to_ear > right_eye_to_ear + 0.03:
                head_text = "Head: LEFT TURN"
            elif right_eye_to_ear > left_eye_to_ear + 0.03:
                head_text = "Head: RIGHT TURN"
            else:
                head_text = "Head: CENTER"

        # 디버깅용 눈/홍채 점
        for idx in [LEFT_EYE_OUTER, LEFT_IRIS_LEFT, LEFT_IRIS_RIGHT,
                    RIGHT_EYE_OUTER, RIGHT_IRIS_LEFT, RIGHT_IRIS_RIGHT]:
            cx, cy = int(lm[idx].x * w), int(lm[idx].y * h)
            cv2.circle(frame, (cx, cy), 3, (0, 255, 255), -1)

    # === 어깨 자세 ===
    shoulder_text = ""
    if pose_result.pose_landmarks:
        lm = pose_result.pose_landmarks.landmark
        left_shoulder_y = lm[LEFT_SHOULDER].y
        right_shoulder_y = lm[RIGHT_SHOULDER].y
        diff = left_shoulder_y - right_shoulder_y

        if diff > 0.012:
            shoulder_text = "LEFT SHOULDER UP"
        elif diff < -0.012:
            shoulder_text = "RIGHT SHOULDER UP"
        else:
            shoulder_text = "SHOULDERS STRAIGHT"

        mp_drawing.draw_landmarks(
            frame,
            pose_result.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

    # 텍스트 출력
    y = 30
    for text in [gaze_text, head_text, shoulder_text]:
        if text:
            cv2.putText(frame, text, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            y += 30

    cv2.imshow("Full Tracking", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

🎥 Gaze + Head + Shoulder Tracking Started - ESC to exit


In [7]:
# 좌우 완성 + 위, 아래 추가

import cv2
import mediapipe as mp
import numpy as np
import time

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

pose = mp_pose.Pose(static_image_mode=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# 랜드마크 인덱스
LEFT_EYE_OUTER = 33
LEFT_IRIS_LEFT = 471
LEFT_IRIS_RIGHT = 469
LEFT_IRIS_TOP = 470
LEFT_IRIS_BOTTOM = 472
LEFT_EYE_CENTER = 468

RIGHT_EYE_OUTER = 263
RIGHT_IRIS_LEFT = 476
RIGHT_IRIS_RIGHT = 474
RIGHT_IRIS_TOP = 475
RIGHT_IRIS_BOTTOM = 477
RIGHT_EYE_CENTER = 473

LEFT_EAR = mp_pose.PoseLandmark.LEFT_EAR
RIGHT_EAR = mp_pose.PoseLandmark.RIGHT_EAR
LEFT_SHOULDER = mp_pose.PoseLandmark.LEFT_SHOULDER
RIGHT_SHOULDER = mp_pose.PoseLandmark.RIGHT_SHOULDER

# 웹캠
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Webcam not opened.")
    exit()

print("🎥 Tracking Started - ESC to exit")

prev_time = time.time()
prev_gaze = "NONE"
gaze_start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    face_result = face_mesh.process(rgb)
    pose_result = pose.process(rgb)

    gaze_text = ""
    head_text = ""
    shoulder_text = ""
    gaze_direction = ""

    if face_result.multi_face_landmarks:
        face = face_result.multi_face_landmarks[0]
        lm = face.landmark

        # === 왼쪽 눈 (좌우/상하) ===
        left_eye_outer = np.array([lm[LEFT_EYE_OUTER].x * w, lm[LEFT_EYE_OUTER].y * h])
        left_iris_left = np.array([lm[LEFT_IRIS_LEFT].x * w, lm[LEFT_IRIS_LEFT].y * h])
        left_iris_right = np.array([lm[LEFT_IRIS_RIGHT].x * w, lm[LEFT_IRIS_RIGHT].y * h])
        left_iris_top = np.array([lm[LEFT_IRIS_TOP].x * w, lm[LEFT_IRIS_TOP].y * h])
        left_iris_bottom = np.array([lm[LEFT_IRIS_BOTTOM].x * w, lm[LEFT_IRIS_BOTTOM].y * h])

        left_iris_width = np.linalg.norm(left_iris_right - left_iris_left) + 1e-6
        left_iris_height = np.linalg.norm(left_iris_top - left_iris_bottom) + 1e-6

        left_ratio_h = np.linalg.norm(left_iris_left - left_eye_outer) / left_iris_width
        left_ratio_v = np.linalg.norm(left_iris_top - left_iris_bottom) / left_iris_height

        # === 오른쪽 눈 (좌우/상하) ===
        right_eye_outer = np.array([lm[RIGHT_EYE_OUTER].x * w, lm[RIGHT_EYE_OUTER].y * h])
        right_iris_left = np.array([lm[RIGHT_IRIS_LEFT].x * w, lm[RIGHT_IRIS_LEFT].y * h])
        right_iris_right = np.array([lm[RIGHT_IRIS_RIGHT].x * w, lm[RIGHT_IRIS_RIGHT].y * h])
        right_iris_top = np.array([lm[RIGHT_IRIS_TOP].x * w, lm[RIGHT_IRIS_TOP].y * h])
        right_iris_bottom = np.array([lm[RIGHT_IRIS_BOTTOM].x * w, lm[RIGHT_IRIS_BOTTOM].y * h])

        right_iris_width = np.linalg.norm(right_iris_right - right_iris_left) + 1e-6
        right_iris_height = np.linalg.norm(right_iris_top - right_iris_bottom) + 1e-6

        right_ratio_h = np.linalg.norm(right_iris_right - right_eye_outer) / right_iris_width
        right_ratio_v = np.linalg.norm(right_iris_top - right_iris_bottom) / right_iris_height

        # 시선 방향 결정
        gaze_h = "LEFT" if left_ratio_h < 0.48 else "RIGHT" if right_ratio_h < 0.48 else "CENTER"
        gaze_v = "UP" if left_ratio_v > 0.55 else "DOWN" if left_ratio_v < 0.45 else "CENTER"
        gaze_direction = f"{gaze_h} / {gaze_v}"

        # 시선 유지 시간 계산
        now = time.time()
        if gaze_direction != prev_gaze:
            duration = now - gaze_start_time
            print(f"👁️ 이전 시선 '{prev_gaze}' 유지 시간: {duration:.2f}s")
            gaze_start_time = now
            prev_gaze = gaze_direction

        gaze_text = f"Gaze: {gaze_direction}"

        # === 고개 방향 ===
        if pose_result.pose_landmarks:
            pose_lm = pose_result.pose_landmarks.landmark
            left_eye_center = lm[LEFT_EYE_CENTER]
            right_eye_center = lm[RIGHT_EYE_CENTER]
            left_ear = pose_lm[LEFT_EAR]
            right_ear = pose_lm[RIGHT_EAR]

            # 좌우
            left_eye_to_ear = abs(left_eye_center.x - left_ear.x)
            right_eye_to_ear = abs(right_eye_center.x - right_ear.x)
            if left_eye_to_ear > right_eye_to_ear + 0.03:
                head_lr = "LEFT"
            elif right_eye_to_ear > left_eye_to_ear + 0.03:
                head_lr = "RIGHT"
            else:
                head_lr = "CENTER"

            # 상하 (눈 → 어깨 중간점 높이 비교)
            nose_y = (left_eye_center.y + right_eye_center.y) / 2
            shoulder_y = (pose_lm[LEFT_SHOULDER].y + pose_lm[RIGHT_SHOULDER].y) / 2
            head_ud = str(nose_y) + str(shoulder_y)

            # if nose_y < shoulder_y - 0.05:
            #     head_ud = "UP"
            # elif nose_y > shoulder_y + 0.05:
            #     head_ud = "DOWN"
            # else:
            #     head_ud = "CENTER"

            head_text = f"Head: {head_lr} / {head_ud}"

    # === 어깨 기울기 ===
    if pose_result.pose_landmarks:
        lm = pose_result.pose_landmarks.landmark
        diff = lm[LEFT_SHOULDER].y - lm[RIGHT_SHOULDER].y
        if diff > 0.012:
            shoulder_text = "LEFT SHOULDER UP"
        elif diff < -0.012:
            shoulder_text = "RIGHT SHOULDER UP"
        else:
            shoulder_text = "SHOULDERS STRAIGHT"

        mp_drawing.draw_landmarks(
            frame,
            pose_result.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

    # 출력
    y = 30
    for text in [gaze_text, head_text, shoulder_text]:
        if text:
            cv2.putText(frame, text, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            y += 30

    cv2.imshow("Gaze + Head + Shoulder (Full)", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


🎥 Tracking Started - ESC to exit
👁️ 이전 시선 'NONE' 유지 시간: 0.33s


In [9]:
# 시선, 고개만 위아래 - chat gpt 수치 그대로 - 고개 엄청 위로 들어야 center.  시선은 아예 안됨.

import cv2
import mediapipe as mp

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)
mp_drawing = mp.solutions.drawing_utils

# 웹캠 열기
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # 이미지 전처리
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    frame_height, frame_width = frame.shape[:2]

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 시선 판단 기준 랜드마크 (왼쪽 눈 기준)
            iris_y = face_landmarks.landmark[468].y
            eye_top_y = face_landmarks.landmark[159].y
            eye_bottom_y = face_landmarks.landmark[145].y

            # 시선 방향 판별
            if iris_y < eye_top_y - 0.005:
                gaze_direction = "👁️ UP"
            elif iris_y > eye_bottom_y + 0.005:
                gaze_direction = "👁️ DOWN"
            else:
                gaze_direction = "👁️ CENTER"

            # 고개 방향 판단 기준 (눈, 코끝, 턱)
            left_eye_y = (face_landmarks.landmark[33].y + face_landmarks.landmark[133].y) / 2
            nose_y = face_landmarks.landmark[1].y
            chin_y = face_landmarks.landmark[152].y

            # 고개 방향 판별
            if nose_y < left_eye_y - 0.02:
                head_direction = "🧑‍🦰 HEAD UP"
            elif nose_y > left_eye_y + 0.02:
                head_direction = "🧑‍🦰 HEAD DOWN"
            else:
                head_direction = "🧑‍🦰 HEAD CENTER"

            # 시각화
            mp_drawing.draw_landmarks(
                frame,
                face_landmarks,
                mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
            )

            cv2.putText(frame, gaze_direction, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            cv2.putText(frame, head_direction, (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 128, 0), 2)

    cv2.imshow('Gaze & Head Direction (Vertical)', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [12]:
# 시선, 고개만 위아래 - chat gpt 수치만 수정 - 잘 안됨
import cv2
import mediapipe as mp

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)
mp_drawing = mp.solutions.drawing_utils

# 웹캠 열기
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # 이미지 전처리
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    frame_height, frame_width = frame.shape[:2]

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 시선 판단 기준 랜드마크 (왼쪽 눈 기준)
            iris_y = face_landmarks.landmark[468].y
            eye_top_y = face_landmarks.landmark[159].y
            eye_bottom_y = face_landmarks.landmark[145].y

            # 시선 방향 판별
            if iris_y < eye_top_y - 0.005:
                gaze_direction = "UP"
            elif iris_y > eye_bottom_y + 0.005:
                gaze_direction = "DOWN"
            else:
                gaze_direction = "CENTER"

            # 고개 방향 판단 기준 (눈, 코끝, 턱)
            left_eye_y = (face_landmarks.landmark[33].y + face_landmarks.landmark[133].y) / 2
            nose_y = face_landmarks.landmark[1].y
            chin_y = face_landmarks.landmark[152].y

            # 고개 방향 판별
            if nose_y < left_eye_y - 0.3:   # 0.4 < 0.8 -0.3 (0.5)
                head_direction = "HEAD UP"
            elif nose_y > left_eye_y - 0.5:   # 0.4 > 0.8 -0.5 (0.3)
                head_direction = "HEAD DOWN"
            else:
                head_direction = "HEAD CENTER"

            # 시각화
            mp_drawing.draw_landmarks(
                frame,
                face_landmarks,
                mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
            )

            cv2.putText(frame, gaze_direction, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            cv2.putText(frame, head_direction, (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 128, 0), 2)

    cv2.imshow('Gaze & Head Direction (Vertical)', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [17]:
# 웹캠 시선 고개 판단 채팅창 -> MediaPipe + solvePnP 조합으로 고개 & 시선 위/아래 실시간 판단하는 완성 코드 작성해줘
# 위아래 고개는 잘됨 -> but 위로는 조금 더 들어야 up, 아래로는 덜 내려도 down 되게. pitch > -15,   pitch > 5
# 시선은 잘 안됨. -> 굉장히 위를 바라볼 때만, 위가 뜸. 아래는 잘 안 됨.
import cv2
import mediapipe as mp
import numpy as np
from math import degrees

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# 얼굴의 3D 모델 좌표 (solvePnP용)
model_points = np.array([
    (0.0, 0.0, 0.0),          # Nose tip - landmark 1
    (0.0, -63.6, -12.5),      # Chin - landmark 152
    (-43.3, 32.7, -26.0),     # Left eye left corner - landmark 33
    (43.3, 32.7, -26.0),      # Right eye right corner - landmark 263
    (-28.9, -28.9, -24.1),    # Left Mouth corner - landmark 78
    (28.9, -28.9, -24.1)      # Right Mouth corner - landmark 308
], dtype=np.float32)

# 카메라 설정 (임의의 내부 파라미터)
def get_camera_matrix(w, h):
    focal_length = w
    center = (w / 2, h / 2)
    return np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype=np.float64)

# 웹캠 열기
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0].landmark

        # 2D 이미지 좌표: solvePnP용
        image_points = np.array([
            [face_landmarks[1].x * w, face_landmarks[1].y * h],     # Nose tip
            [face_landmarks[152].x * w, face_landmarks[152].y * h], # Chin
            [face_landmarks[33].x * w, face_landmarks[33].y * h],   # Left eye left corner
            [face_landmarks[263].x * w, face_landmarks[263].y * h], # Right eye right corner
            [face_landmarks[78].x * w, face_landmarks[78].y * h],   # Left mouth corner
            [face_landmarks[308].x * w, face_landmarks[308].y * h]  # Right mouth corner
        ], dtype=np.float32)

        camera_matrix = get_camera_matrix(w, h)
        dist_coeffs = np.zeros((4, 1))  # 왜곡 무시

        # solvePnP → 고개 회전 벡터
        success, rotation_vector, translation_vector = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs
        )

        # 회전 행렬로 변환
        rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
        pitch = degrees(np.arcsin(-rotation_matrix[2][1]))

        # 고개 위/아래 판단
        if pitch > 5:
            head_direction = "Head Down"
        elif pitch < -15:
            head_direction = "Head Up"
        else:
            head_direction = "Head Center"

        # 시선 위/아래 판단 (왼쪽 눈 기준)
        iris_y = face_landmarks[468].y
        eye_top_y = face_landmarks[159].y
        eye_bottom_y = face_landmarks[145].y
        iris_ratio = (iris_y - eye_top_y) / (eye_bottom_y - eye_top_y)

        if iris_ratio < 0.4:
            gaze_direction = "Gaze Up"
        elif iris_ratio > 0.6:
            gaze_direction = "Gaze Down"
        else:
            gaze_direction = "Gaze Center"

        # 시각화
        cv2.putText(frame, f"{head_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 128, 0), 2)
        cv2.putText(frame, f"{gaze_direction}", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    cv2.imshow('Head & Gaze Pitch Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [15]:
# 웹캠 시선 고개 판단 채팅창 -> MediaPipe + solvePnP 조합으로 고개 & 시선 위/아래 실시간 판단하는 완성 코드 작성해줘
# 위아래 고개는 잘됨 -> 수치 범위 -> 수정
# 시선은 잘 안됨. -> 굉장히 위를 바라볼 때만, 위가 뜸. 아래는 잘 안 됨.
import cv2
import mediapipe as mp
import numpy as np
from math import degrees

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# 얼굴의 3D 모델 좌표 (solvePnP용)
model_points = np.array([
    (0.0, 0.0, 0.0),          # Nose tip - landmark 1
    (0.0, -63.6, -12.5),      # Chin - landmark 152
    (-43.3, 32.7, -26.0),     # Left eye left corner - landmark 33
    (43.3, 32.7, -26.0),      # Right eye right corner - landmark 263
    (-28.9, -28.9, -24.1),    # Left Mouth corner - landmark 78
    (28.9, -28.9, -24.1)      # Right Mouth corner - landmark 308
], dtype=np.float32)

# 카메라 설정 (임의의 내부 파라미터)
def get_camera_matrix(w, h):
    focal_length = w
    center = (w / 2, h / 2)
    return np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype=np.float64)

# 웹캠 열기
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0].landmark

        # 2D 이미지 좌표: solvePnP용
        image_points = np.array([
            [face_landmarks[1].x * w, face_landmarks[1].y * h],     # Nose tip
            [face_landmarks[152].x * w, face_landmarks[152].y * h], # Chin
            [face_landmarks[33].x * w, face_landmarks[33].y * h],   # Left eye left corner
            [face_landmarks[263].x * w, face_landmarks[263].y * h], # Right eye right corner
            [face_landmarks[78].x * w, face_landmarks[78].y * h],   # Left mouth corner
            [face_landmarks[308].x * w, face_landmarks[308].y * h]  # Right mouth corner
        ], dtype=np.float32)

        camera_matrix = get_camera_matrix(w, h)
        dist_coeffs = np.zeros((4, 1))  # 왜곡 무시

        # solvePnP → 고개 회전 벡터
        success, rotation_vector, translation_vector = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs
        )

        # 회전 행렬로 변환
        rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
        pitch = degrees(np.arcsin(-rotation_matrix[2][1]))

        # 고개 위/아래 판단
        if pitch > 15:      # 5 -> 15  (더 고개 들어야 되게)
            head_direction = "Head Down"
        elif pitch < -2:    # -15 -> -2   (덜 고개 숙이지 않아도 되게)
            head_direction = "Head Up"
        else:
            head_direction = "Head Center"

        # 시선 위/아래 판단 (왼쪽 눈 기준)
        iris_y = face_landmarks[468].y
        eye_top_y = face_landmarks[159].y
        eye_bottom_y = face_landmarks[145].y
        iris_ratio = (iris_y - eye_top_y) / (eye_bottom_y - eye_top_y)

        if iris_ratio < 0.4:
            gaze_direction = "Gaze Up"
        elif iris_ratio > 0.6:
            gaze_direction = "Gaze Down"
        else:
            gaze_direction = "Gaze Center"

        # 시각화
        cv2.putText(frame, f"{head_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 128, 0), 2)
        cv2.putText(frame, f"{gaze_direction}", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    cv2.imshow('Head & Gaze Pitch Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
# 시선때문에 gaze tracking 해보려고 했는데, 안돼 -> 설치 어려워서 실패 0.0.8 받기가 힘듬.
import cv2
import time
import numpy as np
from gaze_tracking import GazeTracking
import mediapipe as mp

# GazeTracking 초기화
gaze = GazeTracking()

# MediaPipe FaceMesh 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# 3D 모델 포인트 (solvePnP용)
model_points = np.array([
    (0.0, 0.0, 0.0),          # Nose tip
    (0.0, -63.6, -12.5),      # Chin
    (-43.3, 32.7, -26.0),     # Left eye corner
    (43.3, 32.7, -26.0),      # Right eye corner
    (-28.9, -28.9, -24.1),    # Left mouth
    (28.9, -28.9, -24.1)      # Right mouth
], dtype=np.float32)

# 시선 유지 시간 측정용 변수
gaze_state = None
gaze_start_time = time.time()
gaze_duration_dict = {"UP": 0.0, "DOWN": 0.0, "CENTER": 0.0}

# 카메라 열기
cap = cv2.VideoCapture(0)

def get_camera_matrix(w, h):
    focal_length = w
    center = (w / 2, h / 2)
    return np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype=np.float64)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # GazeTracking 분석
    gaze.refresh(frame)
    vertical_ratio = gaze.vertical_ratio()
    gaze_text = "Gaze: Unknown"

    if vertical_ratio is not None:
        if vertical_ratio < 0.35:
            gaze_direction = "UP"
        elif vertical_ratio > 0.65:
            gaze_direction = "DOWN"
        else:
            gaze_direction = "CENTER"

        gaze_text = f"Gaze: {gaze_direction}"

        # 시선 유지 시간 기록
        now = time.time()
        if gaze_direction != gaze_state:
            if gaze_state:
                gaze_duration_dict[gaze_state] += now - gaze_start_time
            gaze_start_time = now
            gaze_state = gaze_direction

    # MediaPipe로 고개 방향 (pitch) 계산
    results = face_mesh.process(frame_rgb)
    pitch_text = "Head: Unknown"

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0].landmark

        image_points = np.array([
            [face_landmarks[1].x * w, face_landmarks[1].y * h],
            [face_landmarks[152].x * w, face_landmarks[152].y * h],
            [face_landmarks[33].x * w, face_landmarks[33].y * h],
            [face_landmarks[263].x * w, face_landmarks[263].y * h],
            [face_landmarks[78].x * w, face_landmarks[78].y * h],
            [face_landmarks[308].x * w, face_landmarks[308].y * h]
        ], dtype=np.float32)

        camera_matrix = get_camera_matrix(w, h)
        dist_coeffs = np.zeros((4, 1))

        success, rotation_vector, translation_vector = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs
        )

        if success:
            rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
            pitch = np.degrees(np.arcsin(-rotation_matrix[2][1]))

            if pitch < 15:
                pitch_text = "Head: UP"
            elif pitch > -2:
                pitch_text = "Head: DOWN"
            else:
                pitch_text = "Head: CENTER"

    # 결과 출력
    display_text = f"{gaze_text} | {pitch_text}"
    cv2.putText(frame, display_text, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    for i, (k, v) in enumerate(gaze_duration_dict.items()):
        cv2.putText(frame, f"{k}: {v:.1f}s", (30, 100 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow("Gaze & Head Pitch Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 마지막 시선 누적
if gaze_state:
    gaze_duration_dict[gaze_state] += time.time() - gaze_start_time

cap.release()
cv2.destroyAllWindows()

print("\n총 시선 유지 시간:")
for direction, duration in gaze_duration_dict.items():
    print(f"{direction}: {duration:.2f}초")

ModuleNotFoundError: No module named 'gaze_tracking'

In [19]:
!pip install gaze-tracking

In [2]:
# gaze tracking 없앰. 시선 위아래는 mediapipe로 진행해야할 듯. -> 수치 조정중
import cv2
import time
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# 3D 모델 포인트 (solvePnP용)
model_points = np.array([
    (0.0, 0.0, 0.0),
    (0.0, -63.6, -12.5),
    (-43.3, 32.7, -26.0),
    (43.3, 32.7, -26.0),
    (-28.9, -28.9, -24.1),
    (28.9, -28.9, -24.1)
], dtype=np.float32)

# 시선 유지 시간 측정용
gaze_state = None
gaze_start_time = time.time()
gaze_duration_dict = {"UP": 0.0, "DOWN": 0.0, "CENTER": 0.0}

# 카메라 설정
cap = cv2.VideoCapture(0)

def get_camera_matrix(w, h):
    focal_length = w
    center = (w / 2, h / 2)
    return np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype=np.float64)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(frame_rgb)
    gaze_text = "Gaze: Unknown"
    pitch_text = "Head: Unknown"

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        # --- 시선 분석 ---
        left_ratio = (landmarks[468].y - landmarks[159].y) / (landmarks[145].y - landmarks[159].y)
        right_ratio = (landmarks[473].y - landmarks[386].y) / (landmarks[374].y - landmarks[386].y)
        iris_ratio = (left_ratio + right_ratio) / 2

        if iris_ratio < 0.40:
            gaze_direction = "UP"
        elif iris_ratio > 0.60:
            gaze_direction = "DOWN"
        else:
            gaze_direction = "CENTER"

        gaze_text = f"Gaze: {gaze_direction}"

        # 시선 유지 시간 측정
        now = time.time()
        if gaze_direction != gaze_state:
            if gaze_state:
                gaze_duration_dict[gaze_state] += now - gaze_start_time
            gaze_start_time = now
            gaze_state = gaze_direction

        # --- 고개 방향(pitch) 분석 ---
        image_points = np.array([
            [landmarks[1].x * w, landmarks[1].y * h],
            [landmarks[152].x * w, landmarks[152].y * h],
            [landmarks[33].x * w, landmarks[33].y * h],
            [landmarks[263].x * w, landmarks[263].y * h],
            [landmarks[78].x * w, landmarks[78].y * h],
            [landmarks[308].x * w, landmarks[308].y * h]
        ], dtype=np.float32)

        camera_matrix = get_camera_matrix(w, h)
        dist_coeffs = np.zeros((4, 1))

        success, rotation_vector, translation_vector = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs
        )

        if success:
            rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
            pitch = np.degrees(np.arcsin(-rotation_matrix[2][1]))

            if pitch < -15:
                pitch_text = "Head: UP"
            elif pitch > 5:
                pitch_text = "Head: DOWN"
            else:
                pitch_text = "Head: CENTER"

    # --- 결과 출력 ---
    display_text = f"{gaze_text} | {pitch_text}"
    cv2.putText(frame, display_text, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    for i, (k, v) in enumerate(gaze_duration_dict.items()):
        cv2.putText(frame, f"{k}: {v:.1f}s", (30, 100 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow("MediaPipe Gaze & Head Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 마지막 시선 누적
if gaze_state:
    gaze_duration_dict[gaze_state] += time.time() - gaze_start_time

cap.release()
cv2.destroyAllWindows()

print("\n총 시선 유지 시간:")
for direction, duration in gaze_duration_dict.items():
    print(f"{direction}: {duration:.2f}초")


총 시선 유지 시간:
UP: 23.56초
DOWN: 0.00초
CENTER: 30.16초


In [4]:
# 초반 30프레임 찍어서 홍채 위치 비율 평균(avg_ratio) 측정 후 시선 감지 -> 잘 안됨 조금 나아짐
import cv2
import time
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# 3D 모델 포인트 (solvePnP용)
model_points = np.array([
    (0.0, 0.0, 0.0),
    (0.0, -63.6, -12.5),
    (-43.3, 32.7, -26.0),
    (43.3, 32.7, -26.0),
    (-28.9, -28.9, -24.1),
    (28.9, -28.9, -24.1)
], dtype=np.float32)

# 시선 유지 시간 측정용
gaze_state = None
gaze_start_time = time.time()
gaze_duration_dict = {"UP": 0.0, "DOWN": 0.0, "CENTER": 0.0}

# 카메라 설정
cap = cv2.VideoCapture(0)

def get_camera_matrix(w, h):
    focal_length = w
    center = (w / 2, h / 2)
    return np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype=np.float64)

# --- 시선 비율 자동 보정용 ---
calibration_ratios = []
frame_idx = 0
is_calibrated = False
up_thresh = 0.0
down_thresh = 1.0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(frame_rgb)
    gaze_text = "Gaze: Unknown"
    pitch_text = "Head: Unknown"

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        # --- 시선 분석 ---
        left_ratio = (landmarks[468].y - landmarks[159].y) / (landmarks[145].y - landmarks[159].y)
        right_ratio = (landmarks[473].y - landmarks[386].y) / (landmarks[374].y - landmarks[386].y)
        iris_ratio = (left_ratio + right_ratio) / 2

        # 자동 보정 캘리브레이션 (초기 30프레임 평균)
        if not is_calibrated:
            calibration_ratios.append(iris_ratio)
            frame_idx += 1
            if frame_idx == 30:
                avg_ratio = np.mean(calibration_ratios)
                up_thresh = avg_ratio - 0.08
                down_thresh = avg_ratio + 0.08
                is_calibrated = True
                print(f"[Calibration] iris_avg={avg_ratio:.3f}, up<{up_thresh:.3f}, down>{down_thresh:.3f}")
            gaze_direction = "CENTER"
        else:
            if iris_ratio < up_thresh:
                gaze_direction = "UP"
            elif iris_ratio > down_thresh:
                gaze_direction = "DOWN"
            else:
                gaze_direction = "CENTER"

        gaze_text = f"Gaze: {gaze_direction}"

        # 시선 유지 시간 측정
        now = time.time()
        if gaze_direction != gaze_state:
            if gaze_state:
                gaze_duration_dict[gaze_state] += now - gaze_start_time
            gaze_start_time = now
            gaze_state = gaze_direction

        # --- 고개 방향(pitch) 분석 ---
        image_points = np.array([
            [landmarks[1].x * w, landmarks[1].y * h],
            [landmarks[152].x * w, landmarks[152].y * h],
            [landmarks[33].x * w, landmarks[33].y * h],
            [landmarks[263].x * w, landmarks[263].y * h],
            [landmarks[78].x * w, landmarks[78].y * h],
            [landmarks[308].x * w, landmarks[308].y * h]
        ], dtype=np.float32)

        camera_matrix = get_camera_matrix(w, h)
        dist_coeffs = np.zeros((4, 1))

        success, rotation_vector, translation_vector = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs
        )

        if success:
            rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
            pitch = np.degrees(np.arcsin(-rotation_matrix[2][1]))

            if pitch < -15:
                pitch_text = "Head: UP"
            elif pitch > 5:
                pitch_text = "Head: DOWN"
            else:
                pitch_text = "Head: CENTER"

    # --- 결과 출력 ---
    display_text = f"{gaze_text} | {pitch_text}"
    cv2.putText(frame, display_text, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    for i, (k, v) in enumerate(gaze_duration_dict.items()):
        cv2.putText(frame, f"{k}: {v:.1f}s", (30, 100 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow("MediaPipe Gaze & Head Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 마지막 시선 누적
if gaze_state:
    gaze_duration_dict[gaze_state] += time.time() - gaze_start_time

cap.release()
cv2.destroyAllWindows()

print("\n총 시선 유지 시간:")
for direction, duration in gaze_duration_dict.items():
    print(f"{direction}: {duration:.2f}초")

[Calibration] iris_avg=0.401, up<0.321, down>0.481

총 시선 유지 시간:
UP: 29.81초
DOWN: 66.86초
CENTER: 455.62초


In [5]:
# 윗눈꺼풀과 아랫 눈꺼풀의 거리 차이 -> 뜨면 좁아지고, 감으면 길어짐.
import cv2
import time
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# 3D 모델 포인트 (solvePnP용)
model_points = np.array([
    (0.0, 0.0, 0.0),
    (0.0, -63.6, -12.5),
    (-43.3, 32.7, -26.0),
    (43.3, 32.7, -26.0),
    (-28.9, -28.9, -24.1),
    (28.9, -28.9, -24.1)
], dtype=np.float32)

# 시선 유지 시간 측정용
gaze_state = None
gaze_start_time = time.time()
gaze_duration_dict = {"UP": 0.0, "DOWN": 0.0, "CENTER": 0.0}

# 카메라 설정
cap = cv2.VideoCapture(0)

def get_camera_matrix(w, h):
    focal_length = w
    center = (w / 2, h / 2)
    return np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype=np.float64)

# --- 눈꺼풀 기반 자동 보정용 ---
calibration_openings = []
frame_idx = 0
is_calibrated = False
up_thresh = 0.0
down_thresh = 1.0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(frame_rgb)
    gaze_text = "Gaze: Unknown"
    pitch_text = "Head: Unknown"

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        # --- 시선 분석 (눈꺼풀 열림 정도) ---
        eye_top_id = 159
        eye_bottom_id = 145
        iris_id = 468

        eye_top = landmarks[eye_top_id].y
        eye_bottom = landmarks[eye_bottom_id].y
        eye_opening = abs(eye_top - eye_bottom)

        # 캘리브레이션 (초기 30프레임 평균)
        if not is_calibrated:
            calibration_openings.append(eye_opening)
            frame_idx += 1
            if frame_idx == 30:
                avg_opening = np.mean(calibration_openings)
                up_thresh = avg_opening * 0.9
                down_thresh = avg_opening * 1.1
                is_calibrated = True
                print(f"[Calibration] eye_opening_avg={avg_opening:.4f}, up<{up_thresh:.4f}, down>{down_thresh:.4f}")
            gaze_direction = "CENTER"
        else:
            if eye_opening < up_thresh:
                gaze_direction = "DOWN"
            elif eye_opening > down_thresh:
                gaze_direction = "UP"
            else:
                gaze_direction = "CENTER"

        gaze_text = f"Gaze: {gaze_direction}"

        # 시선 유지 시간 측정
        now = time.time()
        if gaze_direction != gaze_state:
            if gaze_state:
                gaze_duration_dict[gaze_state] += now - gaze_start_time
            gaze_start_time = now
            gaze_state = gaze_direction

        # --- 랜드마크 시각화 ---
        def draw_landmark_circle(idx, color):
            cx = int(landmarks[idx].x * w)
            cy = int(landmarks[idx].y * h)
            cv2.circle(frame, (cx, cy), 4, color, -1)

        draw_landmark_circle(eye_top_id, (0, 255, 0))      # 윗눈꺼풀 초록
        draw_landmark_circle(eye_bottom_id, (0, 0, 255))   # 아랫눈꺼풀 빨강
        draw_landmark_circle(iris_id, (255, 0, 0))         # 동공 파랑

        # --- 고개 방향(pitch) 분석 ---
        image_points = np.array([
            [landmarks[1].x * w, landmarks[1].y * h],
            [landmarks[152].x * w, landmarks[152].y * h],
            [landmarks[33].x * w, landmarks[33].y * h],
            [landmarks[263].x * w, landmarks[263].y * h],
            [landmarks[78].x * w, landmarks[78].y * h],
            [landmarks[308].x * w, landmarks[308].y * h]
        ], dtype=np.float32)

        camera_matrix = get_camera_matrix(w, h)
        dist_coeffs = np.zeros((4, 1))

        success, rotation_vector, translation_vector = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs
        )

        if success:
            rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
            pitch = np.degrees(np.arcsin(-rotation_matrix[2][1]))

            if pitch < -15:
                pitch_text = "Head: UP"
            elif pitch > 5:
                pitch_text = "Head: DOWN"
            else:
                pitch_text = "Head: CENTER"

    # --- 결과 출력 ---
    display_text = f"{gaze_text} | {pitch_text}"
    cv2.putText(frame, display_text, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    for i, (k, v) in enumerate(gaze_duration_dict.items()):
        cv2.putText(frame, f"{k}: {v:.1f}s", (30, 100 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow("MediaPipe Gaze & Head Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 마지막 시선 누적
if gaze_state:
    gaze_duration_dict[gaze_state] += time.time() - gaze_start_time

cap.release()
cv2.destroyAllWindows()

print("\n총 시선 유지 시간:")
for direction, duration in gaze_duration_dict.items():
    print(f"{direction}: {duration:.2f}초")

[Calibration] eye_opening_avg=0.0195, up<0.0176, down>0.0215

총 시선 유지 시간:
UP: 57.83초
DOWN: 93.58초
CENTER: 70.07초


In [6]:
# | 조건                                  | 동작                     |
# | ------------------------------------ | ------------------------ |
# | 눈이 거의 닫혔을 때 (`eye_opening < 0.015`) | 깜빡임으로 판단 → 이전 시선 상태 유지 |
# | 깜빡임 직후 일정 프레임(`5 frame`) 동안      | 여전히 이전 시선 상태 유지 (쿨다운)  |
# -> 괜찮게 나오는 편 -> 수치 좀 조절해야할 듯

import cv2
import time
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)

# 3D 모델 포인트 (solvePnP용)
model_points = np.array([
    (0.0, 0.0, 0.0),
    (0.0, -63.6, -12.5),
    (-43.3, 32.7, -26.0),
    (43.3, 32.7, -26.0),
    (-28.9, -28.9, -24.1),
    (28.9, -28.9, -24.1)
], dtype=np.float32)

# 시선 유지 시간 측정용
gaze_state = None
gaze_start_time = time.time()
gaze_duration_dict = {"UP": 0.0, "DOWN": 0.0, "CENTER": 0.0}

# 카메라 설정
cap = cv2.VideoCapture(0)

def get_camera_matrix(w, h):
    focal_length = w
    center = (w / 2, h / 2)
    return np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype=np.float64)

# --- 눈꺼풀 기반 자동 보정용 ---
calibration_openings = []
frame_idx = 0
is_calibrated = False
up_thresh = 0.0
down_thresh = 1.0

# --- 깜빡임 감지용 ---
blink_threshold = 0.015
blink_cooldown_frames = 5
blink_cooldown_counter = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(frame_rgb)
    gaze_text = "Gaze: Unknown"
    pitch_text = "Head: Unknown"

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        # --- 시선 분석 (눈꺼풀 열림 정도) ---
        eye_top_id = 159
        eye_bottom_id = 145
        iris_id = 468

        eye_top = landmarks[eye_top_id].y
        eye_bottom = landmarks[eye_bottom_id].y
        eye_opening = abs(eye_top - eye_bottom)

        # 깜빡임 처리
        if eye_opening < blink_threshold:
            blink_cooldown_counter = blink_cooldown_frames
            gaze_direction = gaze_state  # 깜빡일 때는 이전 상태 유지
        elif blink_cooldown_counter > 0:
            blink_cooldown_counter -= 1
            gaze_direction = gaze_state  # 쿨다운 동안도 유지
        else:
            # 캘리브레이션 (초기 30프레임 평균)
            if not is_calibrated:
                calibration_openings.append(eye_opening)
                frame_idx += 1
                if frame_idx == 30:
                    avg_opening = np.mean(calibration_openings)
                    up_thresh = avg_opening * 1.1
                    down_thresh = avg_opening * 0.9
                    is_calibrated = True
                    print(f"[Calibration] eye_opening_avg={avg_opening:.4f}, up>{up_thresh:.4f}, down<{down_thresh:.4f}")
                gaze_direction = "CENTER"
            else:
                if eye_opening > up_thresh:
                    gaze_direction = "UP"
                elif eye_opening < down_thresh:
                    gaze_direction = "DOWN"
                else:
                    gaze_direction = "CENTER"

        gaze_text = f"Gaze: {gaze_direction}"

        # 시선 유지 시간 측정
        now = time.time()
        if gaze_direction != gaze_state:
            if gaze_state:
                gaze_duration_dict[gaze_state] += now - gaze_start_time
            gaze_start_time = now
            gaze_state = gaze_direction

        # --- 랜드마크 시각화 ---
        def draw_landmark_circle(idx, color):
            cx = int(landmarks[idx].x * w)
            cy = int(landmarks[idx].y * h)
            cv2.circle(frame, (cx, cy), 4, color, -1)

        draw_landmark_circle(eye_top_id, (0, 255, 0))      # 윗눈꺼풀 초록
        draw_landmark_circle(eye_bottom_id, (0, 0, 255))   # 아랫눈꺼풀 빨강
        draw_landmark_circle(iris_id, (255, 0, 0))         # 동공 파랑

        # --- 고개 방향(pitch) 분석 ---
        image_points = np.array([
            [landmarks[1].x * w, landmarks[1].y * h],
            [landmarks[152].x * w, landmarks[152].y * h],
            [landmarks[33].x * w, landmarks[33].y * h],
            [landmarks[263].x * w, landmarks[263].y * h],
            [landmarks[78].x * w, landmarks[78].y * h],
            [landmarks[308].x * w, landmarks[308].y * h]
        ], dtype=np.float32)

        camera_matrix = get_camera_matrix(w, h)
        dist_coeffs = np.zeros((4, 1))

        success, rotation_vector, translation_vector = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs
        )

        if success:
            rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
            pitch = np.degrees(np.arcsin(-rotation_matrix[2][1]))

            if pitch < -15:
                pitch_text = "Head: UP"
            elif pitch > 5:
                pitch_text = "Head: DOWN"
            else:
                pitch_text = "Head: CENTER"

    # --- 결과 출력 ---
    display_text = f"{gaze_text} | {pitch_text}"
    cv2.putText(frame, display_text, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    for i, (k, v) in enumerate(gaze_duration_dict.items()):
        cv2.putText(frame, f"{k}: {v:.1f}s", (30, 100 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow("MediaPipe Gaze & Head Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 마지막 시선 누적
if gaze_state:
    gaze_duration_dict[gaze_state] += time.time() - gaze_start_time

cap.release()
cv2.destroyAllWindows()

print("\n총 시선 유지 시간:")
for direction, duration in gaze_duration_dict.items():
    print(f"{direction}: {duration:.2f}초")

[Calibration] eye_opening_avg=0.0212, up>0.0234, down<0.0191

총 시선 유지 시간:
UP: 34.96초
DOWN: 47.14초
CENTER: 47.76초
